In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
pd.options.display.max_columns = None

In [ ]:
target_source = '../../DATA/JSON/PAPAER_SCRAPING.json'
data = []

with open(target_source) as d:
    data = json.load(d)

In [ ]:
len(data)

In [ ]:
data[0]

In [ ]:
idx_terms_list = []
bucket1 = []
bucket2 = []
valid_data = []

In [ ]:
for i in data:
    if (i['data']['Document_type'][-1] == '•'):
        i['data']['Document_type'] = i['data']['Document_type'][:-1]
    
    if (i['data']['Document_type'] == 'Article'):
        bucket1.append(i)

In [ ]:
len(bucket1)

In [ ]:
for i in bucket1:
    if (len(i['data']['idxterms']) == 0 or (len(i['data']['idxterms']) != 0 and i['data']['idxterms'][0] != '')):
        bucket2.append(i)

In [ ]:
len(bucket2)

In [ ]:
for i in bucket2:
    if (len(i['data']['authkeyword']) == 0 or (len(i['data']['authkeyword']) != 0 and i['data']['authkeyword'][0] != '')):
        valid_data.append(i)

In [ ]:
len(valid_data)

In [ ]:
authkeyword_list = []
idxterms_list = []
agency_list = []
funding_status_cite_list = []

In [ ]:
for i in valid_data:
    eid = i['data']['Eid']
    authkeyword = i['data']['authkeyword']
    idxterms = i['data']['idxterms']
    cite_count = i['data']['Cited_Count']
    funding_agency = i['data']['Funding_Agency']

    for i in authkeyword:
        authkeyword_list.append([eid, i])

    for i in idxterms:
        idxterms_list.append([eid, i])

    for i in funding_agency:
        agency_id = None
        if (i['link'] != None):
            agency_id = i['link'].split('/')[-1]
        if (agency_id != None):
            agency_id = str(agency_id)
        agency_list.append([eid, i['name'], agency_id])

    funding_status_cite_list.append([eid, len(funding_agency), cite_count, (0 if len(funding_agency) == 0 else 1), (0 if cite_count == 0 else 1) ])

In [ ]:
authkeyword_df = pd.DataFrame(authkeyword_list, columns=['Eid', 'Auth_Keyword']).drop_duplicates().dropna()

In [ ]:
idxterms_df = pd.DataFrame(idxterms_list, columns=['Eid', 'Index_Terms']).dropna() #ยังไม่ได้ drop duplicate
idxterms_df

In [ ]:
funding_status_cite_df = pd.DataFrame(funding_status_cite_list, columns=['Eid', 'Funding_Count', 'Cite-By_Count', 'Has_Funding', 'Has_Cite-By']).drop_duplicates().dropna()
funding_status_cite_df

In [ ]:
funding_status_cite_df.to_csv('scrape_funding_cite.csv', index=False)

In [ ]:
agency_list

In [ ]:
agency_df = pd.DataFrame(agency_list, columns=['Eid', 'Agency_Name', 'Agency_ID'])
agency_df

In [ ]:
agency_country_data = pd.read_csv('agency_data.csv').drop(columns='Unnamed: 0')
agency_country_data = agency_country_data.rename(columns={'Agency_Name' : 'Agency_Name_From_Data'})
agency_country_data['Agency_ID'] = agency_country_data['Agency_ID'].apply(lambda x : str(x))

In [ ]:
agency_df = agency_df.merge(agency_country_data, how='left', on='Agency_ID')
agency_df = agency_df.drop(columns='Agency_Name_From_Data')

agency_df

In [ ]:
agency_df[agency_df['Agency_ID'].isna()]['Agency_Name'].drop_duplicates().to_csv('funding_name_list_scrape.csv', index=False)

In [ ]:
agency_df.to_csv('funding_list_by_eid_scrape.csv', index=False)